In [1]:
import numpy as np
import pandas as pd

---

### general settings

In [2]:
# year (as a string)
yr = "2099"

# registered students file
registered = "./raw_data/%s/corso_eccellenza_iscritti.csv" % yr

# list of files from different meetings, in chronological order
attendees = {
    "2099" : [
        "./raw_data/%s/participants_01234567890_1.csv" % yr,  # meeting 0
        "./raw_data/%s/participants_01234567890_2.csv" % yr,  # meeting 1
        "./raw_data/%s/participants_01234567890_3.csv" % yr,  # meeting 2
    ],
}

# minimum duration for "long" presence
nMins = 10

# list of codici meccanografici and corresponding institution names & locations (format: Name_Location)
locations = {
    'COPC020007' : "Volta_Como", 
    'FEPC020005' : "Ariosto_Ferrara", 
    'SS19510' : "",
}

---

### data input and processing

In [3]:
# load all the data
df_attendees = {}
if yr=="2021":
    df_registered = pd.read_csv(registered, sep=",", names=["surname", "firstName", "institution"])
    for i, iName in enumerate(attendees[yr]):
        df_attendees[i] = pd.read_csv(iName, sep=",", skiprows=1, names=["nickname", "mail", "t0", "t1", "duration", "guest", "consent"])
        df_attendees[i] = df_attendees[i].drop(columns=["t0", "t1", "guest", "consent"])
else:
    df_registered = pd.read_csv(registered, sep=",", names=["timestamp", "mail_comms", "firstName", "surname", "nickname", "mail_zoom", "institution", "year"])
    for i, iName in enumerate(attendees[yr]):
        df_attendees[i] = pd.read_csv(iName, sep=",", skiprows=4, names=["nickname", "mail", "t0", "t1", "duration", "guest"])
        df_attendees[i] = df_attendees[i].drop(columns=["t0", "t1", "guest"])
print("registered entries: %d" % df_registered.shape[0])

# data preparation -- df_registered
if yr=="2021":
    for s in [col for col in df_registered.columns.unique() if col!="institution"]:
        df_registered[s] = df_registered[s].str.lower()
        df_registered[s] = df_registered[s].fillna("")
        df_registered[s] = df_registered[s].str.replace("\t", " ")
        if s=="firstName":
            originalNamesBak = df_registered[s].copy()  # make a backup of the original names (cleaned fron nan and \t signs) before removing the accents
        df_registered[s] = df_registered[s].str.replace("à", "a").str.replace("è", "e").str.replace("ì", "i").str.replace("ò", "o").str.replace("ù", "u")
    df_registered["firstName1"] = pd.Series([s[0] if len(s)>0 else "" for s in df_registered["firstName"].str.split()])  # first of the names in the first name
    df_registered["firstName2"] = pd.Series([s[1] if len(s)>1 else "~NONE~" for s in df_registered["firstName"].str.split()])  # second of the names in the first name
else:
    for s in [col for col in df_registered.columns.unique() if (col=="firstName")|(col=="surname")|(col=="nickname")|(col=="email_zoom")]:
        df_registered[s] = df_registered[s].str.lower()
        df_registered[s] = df_registered[s].fillna("")
        df_registered[s] = df_registered[s].str.replace("\t", " ")
        df_registered[s] = df_registered[s].str.rstrip()
        if s=="firstName":
            originalNamesBak = df_registered[s].copy()  # make a backup of the original names (cleaned fron nan and \t signs) before removing the accents
        if s=="nickname":
            originalNicknamesBak = df_registered[s].copy()  # make a backup of the original names (cleaned fron nan and \t signs) before removing the accents
        df_registered[s] = df_registered[s].str.replace("à", "a").str.replace("è", "e").str.replace("ì", "i").str.replace("ò", "o").str.replace("ù", "u")
    df_registered["firstName1"] = pd.Series([s[0] if len(s)>0 else "" for s in df_registered["firstName"].str.split()])  # first of the names in the first name
    df_registered["firstName2"] = pd.Series([s[1] if len(s)>1 else "~NONE~" for s in df_registered["firstName"].str.split()])  # second of the names in the first name
    df_registered["institution"] = df_registered["institution"].str.upper()  # institution codes are all-uppercase
    
# data preparation -- df_attendees[X]
for i in range(len(attendees[yr])):
    for s in [s0 for s0 in df_attendees[i].columns.unique() if not (s0=="duration")]:
        df_attendees[i][s] = df_attendees[i][s].str.lower()
        df_attendees[i][s] = df_attendees[i][s].str.replace("à", "a").str.replace("è", "e").str.replace("ì", "i").str.replace("ò", "o").str.replace("ù", "u")

registered entries: 11


**settings** on manual nickname tweaking in attendees data:

In [4]:
# manual nickname tweaking (all lowercase)
if yr=="2099":
    for i in range(len(attendees[yr])):
        df_attendees[i].loc[df_attendees[i].nickname=="succo (succo)", "nickname"] = df_attendees[i].nickname.apply(lambda s: s.replace("succo (succo)", "strosprobat"))
        df_attendees[i].loc[df_attendees[i].nickname=="iphone di john", "nickname"] = df_attendees[i].nickname.apply(lambda s: s.replace("iphone di john", "titor john"))

In [5]:
# create all possible registered name combinations -- basic, more complex cases below...
df_registered["surname-firstName"] = df_registered.surname+" "+df_registered.firstName  # rossi mario luigi
df_registered["firstName-surname"] = df_registered.firstName+" "+df_registered.surname  # mario luigi rossi
df_registered["surnamefirstName"] = df_registered.surname.str.replace(" ", "")+df_registered.firstName.str.replace(" ", "")  # rossimarioluigi
df_registered["firstNamesurname"] = df_registered.firstName.str.replace(" ", "")+df_registered.surname.str.replace(" ", "")  # marioluigirossi

df_registered["surname-firstName1"] = df_registered.surname+" "+df_registered.firstName1  # rossi mario
df_registered["firstName1-surname"] = df_registered.firstName1+" "+df_registered.surname  # mario rossi
df_registered["surnamefirstName1"] = df_registered.surname.str.replace(" ", "")+df_registered.firstName1.str.replace(" ", "")  # rossimario
df_registered["firstName1surname"] = df_registered.firstName1.str.replace(" ", "")+df_registered.surname.str.replace(" ", "")  # mariorossi

df_registered["surname-firstName2"] = df_registered.surname+" "+df_registered.firstName2  # rossi luigi
df_registered["firstName2-surname"] = df_registered.firstName2+" "+df_registered.surname  # luigi rossi
df_registered["surnamefirstName2"] = df_registered.surname.str.replace(" ", "")+df_registered.firstName2.str.replace(" ", "")  # rossiluigi
df_registered["firstName2surname"] = df_registered.firstName2.str.replace(" ", "")+df_registered.surname.str.replace(" ", "")  # luigirossi

In [6]:
# function to find presence to each meeting...
if yr=="2021":
    def check_presence(df_registered, df_attendees, i, nMins):
        dfTemp = pd.DataFrame()
        compare = list(df_attendees[i].nickname.unique()) if (nMins is None) else list(df_attendees[i][df_attendees[i].duration > nMins].nickname.unique())
        compare = [s0 for s0 in compare if str(s0) != "nan"]
        for s in [s0 for s0 in df_registered.columns.unique() if ((s0!="firstName") & (s0!="firstName1") & (s0!="firstName2") & (s0!="surname") & (s0!="institution") & (not ("presence" in s0)))]:
            dfTemp[s] = df_registered[s].apply(lambda x: any([str(x) in x0 for x0 in compare]))  # previously defined case looked for in list of attendees

        # more particular cases: "mario luigi" & "rossi" inside more complicated nickname string
        dfTemp["firstNameANDsurname"] = df_registered.apply(lambda x: any([(str(x.surname) in x0 and str(x.firstName) in x0) for x0 in compare]), axis=1)
        # more particular cases: "mario" & "rossi" inside more complicated nickname string
        dfTemp["firstName1ANDsurname"] = df_registered.apply(lambda x: any([(str(x.surname) in x0 and str(x.firstName1) in x0) for x0 in compare]), axis=1)
        # more particular cases: "luigi" & "rossi" inside more complicated nickname string
        dfTemp["firstName2ANDsurname"] = df_registered.apply(lambda x: any([(str(x.surname) in x0 and str(x.firstName2) in x0) for x0 in compare]), axis=1)
        # more particular cases: "mario" & "luigi" & "rossi" inside more complicated nickname string
        dfTemp["firstName1ANDfirstName2ANDsurname"] = df_registered.apply(lambda x: any([(str(x.surname) in x0 and str(x.firstName1) and str(x.firstName2) in x0) for x0 in compare]), axis=1)
        # more particular cases: "m." & "rossi" inside more complicated nickname string
        dfTemp["firstName1ANDsurname_alsoInitials"] = df_registered.apply(lambda x: any([(str(x.surname) in x0 and (str(x.firstName1)[0]+"." if len(x.firstName1)>0 else "") in x0) for x0 in compare]), axis=1)
        # more particular cases: "l." & "rossi" inside more complicated nickname string
        dfTemp["firstName2ANDsurname_alsoInitials"] = df_registered.apply(lambda x: any([(str(x.surname) in x0 and (str(x.firstName2)[0]+"." if len(x.firstName2)>0 else "") in x0) for x0 in compare]), axis=1)
        # more particular cases: "m." & "l." & "rossi" inside more complicated nickname string
        dfTemp["firstName1ANDfirstName2ANDsurname_alsoInitials"] = df_registered.apply(lambda x: any([(str(x.surname) in x0 and (str(x.firstName1)[0]+"." if len(x.firstName1)>0 else "") and (str(x.firstName2)[0]+"." if len(x.firstName2)>0 else "") in x0) for x0 in compare]), axis=1)

        outSeries = dfTemp.apply(any, axis=1)  # at least 1 case has to be true in the list of attendees
        return outSeries, dfTemp
else:
    def check_presence(df_registered, df_attendees, i, nMins):
        dfTemp = pd.DataFrame()
        compare_nickname = list(df_attendees[i].nickname.unique()) if (nMins is None) else list(df_attendees[i][df_attendees[i].duration > nMins].nickname.unique())
        compare_nickname = [s0 for s0 in compare_nickname if str(s0) != "nan"]
        compare_mail = list(df_attendees[i].mail.unique()) if (nMins is None) else list(df_attendees[i][df_attendees[i].duration > nMins].mail.unique())
        compare_mail = [s0 for s0 in compare_mail if str(s0) != "nan"]
        dfTemp["nickname"] = df_registered["nickname"].apply(lambda x: any([str(x) in x0 for x0 in compare_nickname]))
        dfTemp["mail"] = df_registered["mail_zoom"].apply(lambda x: any([str(x) in x0 for x0 in compare_mail]))
        
        # more particular cases: "mario luigi" & "rossi" inside more complicated nickname string
        dfTemp["firstNameANDsurname"] = df_registered.apply(lambda x: any([(str(x.surname) in x0 and str(x.firstName) in x0) for x0 in compare_nickname]), axis=1)

        outSeries = dfTemp.apply(any, axis=1)  # at least 1 case has to be true in the list of attendees
        return outSeries, dfTemp

# ...applied to all the meetings
for i in range(len(attendees[yr])):
    df_registered["presence_"+str(i)], _ = check_presence(df_registered, df_attendees, i, None)
    df_registered["presence_long_"+str(i)], _ = check_presence(df_registered, df_attendees, i, nMins)

In [7]:
# create output dataframe
df_output0 = pd.DataFrame(originalNicknamesBak, columns=["nickname"])
df_output1 = pd.DataFrame(originalNamesBak, columns=["firstName"])
df_output2 = pd.DataFrame(df_registered, columns=["surname", "institution"]+["presence_"+str(i) for i in range(len(attendees[yr]))]+["presence_long_"+str(i) for i in range(len(attendees[yr]))])
df_output = pd.concat([df_output0.reset_index(drop=True), df_output1.reset_index(drop=True), df_output2.reset_index(drop=True)], axis=1)

# re-capitalise all the names
df_output["firstName"] = df_output["firstName"].str.title()
df_output["surname"] = df_output["surname"].str.title()

# from 2022 on: check for duplication conflicts and solve them
if yr!="2021":
    df_output["isDuplicate"] = df_output[["firstName", "surname", "institution"]].duplicated(keep=False)
    lsDuplicated = np.array([df_output[df_output.isDuplicate].firstName.to_list(), df_output[df_output.isDuplicate].surname.to_list(), df_output[df_output.isDuplicate].institution.to_list()]).T.tolist()
    lsDuplicated = list(map(list, set(map(lambda i: tuple(i), lsDuplicated))))
    print("found %d multiple entries among registered students!" % len(lsDuplicated))
    for duplicatedEntry in lsDuplicated:
        df_output_temp = df_output[(df_output.firstName==duplicatedEntry[0]) & (df_output.surname==duplicatedEntry[1]) & (df_output.institution==duplicatedEntry[2])]
        df_output_sol = df_output_temp.iloc[0].to_frame().transpose()
        for presenceCol in [s for s in df_output_temp.columns if "presence_" in s]:
            df_output_sol[presenceCol] = any(df_output_temp[presenceCol])
        df_output_sol["isDuplicate"] = False
        df_output = pd.concat([df_output, df_output_sol])
    df_output = df_output[~df_output.isDuplicate]
    df_output = df_output.reset_index(drop=True)
    print("registered entries without multiples: %d" % df_output.shape[0])

found 1 multiple entries among registered students!
registered entries without multiples: 10


In [8]:
# replace codice meccanografico with institution name & location
if yr!="2021":
    for s in df_output.institution.unique():
        if ((s in locations.keys()) & (locations[s] != "")):
            df_output.loc[df_output.institution==s, "institution_full"] = locations[s]
        else:
            df_output.loc[df_output.institution==s, "institution_full"] = "LOCATION_MISSING/%s" % s
            print("warning: %s not in list of institution locations or empty entry!" % s)
            
lsLocationsFinal = df_output["institution" if yr=="2021" else "institution_full"].unique()
print("registered institutions: %d (%d are LOCATION_MISSING)" % (len(lsLocationsFinal), len([s for s in lsLocationsFinal if "LOCATION_MISSING" in s])))

registered institutions: 3 (1 are LOCATION_MISSING)


---

### single-meeting (online) analysis

list of registered students absent at a certain meeting (selectable in the cell)

In [9]:
iMeeting = None  # index of the selected meeting; if None, skip this cell

if not (iMeeting is None):
    print("ABSENT AT LECTURE %d:" % iMeeting)
    dfBool = df_output["presence_%s" % iMeeting]==False
    for i, s in enumerate(df_output[dfBool].surname+" "+df_output[dfBool].firstName):
        print("%3d" % i, s)
else:
    print("no lecture selected, skipping this...")

no lecture selected, skipping this...


**output file** list of presences (for >10 minutes) at a certain meeting &mdash; done for all meetings

In [10]:
for iMeeting in range(len(attendees[yr])):
    nameMaxWidths = (df_output["surname"].apply(lambda x : len(x)).max() + 2, df_output["surname"].apply(lambda x : len(x)).max() + 3)
    with open("./out_data/%s/presences_lecture%s.txt" % (yr, iMeeting), "w+") as f:
        for i in df_output.index:
            rowName = (df_output["surname"][i]+",", df_output["firstName"][i]+",")
            print("%4d," % i, "".join((val.ljust(width) for val, width in zip(rowName[0:2], nameMaxWidths))), "" if df_output["presence_long_%s" % iMeeting][i] else "absent", file=f)
    print("presence file written for lecture %s" % iMeeting)

presence file written for lecture 0
presence file written for lecture 1
presence file written for lecture 2


---

### final analysis

number of lectures attended

also **setting** manual tweaking to output data (only nLectures):

In [11]:
# add nr. of lectures attended
lsPresences = [s for s in df_output.columns if "presence_long_" in s]
df_output["nLectures"] = df_output[lsPresences].sum(axis=1)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# manual tweakings to output -- nLectures
if yr=="2099":
    df_output.loc[df_output.surname=="Bini", "nLectures"] = 2  # they're BINNNNNNI in the 1st lecture
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

for i in range(len(attendees[yr])+1):
    print("nr. of attenders to %d lectures = %d" % (i, df_output[df_output.nLectures == i].shape[0]))

nr. of attenders to 0 lectures = 1
nr. of attenders to 1 lectures = 0
nr. of attenders to 2 lectures = 5
nr. of attenders to 3 lectures = 4


**output file** list of attenders to 0 lectures

In [12]:
if yr=="2021":
    with open("./out_data/%s/attenders_zero_lectures.txt" % yr, "w+") as f:
        for i in df_output.index:
            if df_output.nLectures[i]==0:
                print("%4d," % i, df_output["surname"][i]+",", df_output["firstName"][i]+",", df_output["institution"][i], file=f)
else:
    with open("./out_data/%s/attenders_zero_lectures.txt" % yr, "w+") as f:
        for i in df_output.index:
            if df_output.nLectures[i]==0:
                print("%4d," % i, df_output["surname"][i]+",", df_output["firstName"][i]+",", df_output["nickname"][i]+",", df_output["institution_full"][i], file=f)
print("list of attenders to 0 lectures written")

list of attenders to 0 lectures written


**output file** number of lectures (and hours) attended per registered name (only those who attended to at least 1 lecture) &mdash; specific format for diplomas.ipynb

In [13]:
nameMaxWidths = (df_output["surname"].apply(lambda x : len(x)).max() + 1, df_output["surname"].apply(lambda x : len(x)).max() + 2)
with open("./out_data/%s/presences_total.txt" % yr, "w+") as f:
    for i in df_output.index:
        if df_output["nLectures"][i]>0:
            print("%4d," % i, df_output["surname"][i]+",", df_output["firstName"][i]+",", df_output["institution" if yr=="2021" else "institution_full"][i]+",", str(df_output["nLectures"][i])+",", 2*df_output["nLectures"][i], file=f)
print("final presence file written")

final presence file written
